## ABCD method

### calculate cb,cc,cd

In [3]:
import awkward as ak
import uproot
import hist
import os
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.base import NanoEventsArray
from imp import reload #每次修改调用函数后可以更新
import correctionlib, rich
import matplotlib.pyplot as plt
import sys
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

/tmp/ipykernel_51032/4119456318.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload #每次修改调用函数后可以更新


### 录入文件

In [4]:
def load_event_dict(file_prefix, keys):
    data_subkeys = ['photon_pt','Photon_genPartFlav','Gen_statusflag','photon_endcap','photon_barrel']#'dr_lg','photon_endcap','photon_barrel'
    other_subkeys = data_subkeys + ['generator_weight', 'event_weight']
    
    event_dict = {}
    for key in keys:
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        event_dict[key] = {}
        for subkey in subkeys:
            event_dict[key][subkey] = ak.from_parquet(f'{file_prefix}_{key}_{subkey}.parquet')
    return event_dict

In [5]:
keys = ['zg0', 'zg1', 'zg2','zg3','tt','zz','wz','ww','dy2','dy']  #
# 读取数据
data_A = load_event_dict('ABCD_file/closure_test/A/event_final', keys)
data_B = load_event_dict('ABCD_file/closure_test/B/event_final', keys)
data_C = load_event_dict('ABCD_file/closure_test/C/event_final', keys)
data_D = load_event_dict('ABCD_file/closure_test/D/event_final', keys)
# event_A = load_event_dict('test_file/A/event_final', keys)


In [25]:
LUMI = {
    "2016pre": 19.52, "2016post": 16.81, "2017": 41.48, "2018": 59.83, "2022CD": 6.36,
    "2022FG": 21.6,"2022EFG": 39.22
}
XSEC = {
    'zg0': 124.6,
    'zg1': 1.74,
    'zg2': 0.30,
    'zg3': 0.042,
    'tt': 96.9,
    'ww': 116.8,
    'wz': 54.3,
    'zz': 16.7,
    'dy': 5558.0,
    'dy1':19317.5,
    'dy2':6346.0,
}

# 定义方程
def define_equation():
    NA_prompt, NA, NA_bkg, R, NB, cB, NA_prompt, NB_bkg, NC, cC, ND, cD, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NA_prompt NB_bkg NC cC ND cD NC_bkg ND_bkg')
    equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))
    return equation, NA_prompt

# 解方程并输出结果
def solve_equation(values):
    equation, NA_prompt = define_equation()
    solution = sp.solve(equation.subs(values), NA_prompt)
    return solution

# 提示率值
prompt_values = {
    'end': {
        'A': 1735.307602942849, 'B': 463.3046431337174, 'C': 55.456196229913594, 'D': 21.18293413012198,
    },
    'barrel': {
        'A': 4724.191909791624, 'B': 1249.6579314426272, 'C': 20.423137090023083, 'D': 5.054112003575507,
    }
}

keys = ['tt','wz','dy2']
datasets = {'data_A': data_A, 'data_B': data_B, 'data_C': data_C, 'data_D': data_D}

# 新增一个循环来处理barrel和endcap的情况
for part_type in ['barrel', 'end']:
    print(f"\nProcessing for {part_type.upper()}:")

    values = {
        R: 1.3210 if part_type == 'barrel' else 1.177103,
        cB: 0.264648770 if part_type == 'barrel' else  0.2668905,
        cC: 0.0043251 if part_type == 'barrel' else 0.0319460,
        cD: 0.0010703 if part_type == 'barrel' else 0.012202605,
        NA_bkg: 182.88692 if part_type == 'barrel' else 78.75595088454901,
        NB_bkg: 45.410448625775594,
        NC_bkg: 0.7525415693858519 if part_type == 'barrel' else 2.793944608044445,
        ND_bkg: 0.18241031668551638 if part_type == 'barrel' else 0.9745339205164781,
    }
    for dataset_name, dataset in datasets.items():
        total_count_all = 0
        total_count_prompt = 0
        total_count_non = 0
        print(f"\nFor {dataset_name}:")

        # 使用新的prompt_values字典来获取提示率值
        dataset_prompt = prompt_values[part_type][dataset_name[-1]]

        for key in keys:
            photon_data = dataset[key]
            if len(photon_data['Photon_genPartFlav']) != 0:
                # 根据part_type选择masked_photon
                if part_type == 'barrel':
                    masked_photon = photon_data['Photon_genPartFlav'][photon_data['photon_barrel']]
                else:
                    masked_photon = photon_data['Photon_genPartFlav'][photon_data['photon_endcap']]

                filtered_photon = ak.flatten(masked_photon)
                count_1 = ak.sum(filtered_photon == 1)
                count_not_1 = ak.sum(filtered_photon != 1)
                count_all = ak.sum(filtered_photon != -1)

                event_weight_sign_sum = ak.sum(np.sign(photon_data['event_weight']))
                true_prompt = count_1 * XSEC[key] * LUMI['2022FG'] * 1e3 / event_weight_sign_sum
                true_non = count_not_1 * XSEC[key] * LUMI['2022FG'] * 1e3 / event_weight_sign_sum
                true_all = count_all * XSEC[key] * LUMI['2022FG'] * 1e3 / event_weight_sign_sum

                if 'dy' in key:  # 对于含 'dy' 的样本
                    total_count_all += true_non
                else:  # 对于其他样本
                    total_count_all += true_all
                    total_count_prompt += true_prompt
                total_count_non += true_non
                
                print(f"\tFor {key}:")
                print(f"\tCount of prompt: {true_prompt}")
                print(f"\tCount of nonprompt: {true_non}")
                print(f"\tCount of all: {true_all}")
                print("\t------")

    total_count_all += dataset_prompt
    total_count_prompt += dataset_prompt
    values[locals()[f"N{dataset_name[-1]}"]] = total_count_all
    print(values)

    # 解方程并打印结果
    if part_type == 'barrel':
        values.update({
            NA: NA, 
            NB: NB,
            NC: NC,
            ND: ND,
        })
        # values_barrel[locals()[f"N{dataset_name[-1]}"]] = total_count_all
        # solution = solve_equation(values_barrel)
        # print(f"NA_prompt_barrel = {solution[0]}")
        # print(f"NA_prompt_barrel = {solution[1]}")
    else:
        values.update({
            NA: NA, 
            NB: NB,
            NC: NC,
            ND: ND,
        })
        # values_endcap[locals()[f"N{dataset_name[-1]}"]] = total_count_all
        # solution = solve_equation(values_endcap)
        # print(f"NA_prompt_endcap = {solution[0]}")
        # print(f"NA_prompt_endcap = {solution[1]}")
    solution = solve_equation(values)
    print(f"NA_prompt_{part_type} = {solution[0]}")
    print(f"NA_prompt_{part_type} = {solution[1]}")
    
    # 打印该数据集的所有样本的 count_all 和 count_prompt 总和
    print(f"\tTotal count_all for {dataset_name}: {total_count_all}")
    print(f"\tTotal count_prompt for {dataset_name}: {total_count_prompt}")
    print(f"\tTotal count_non for {dataset_name}: {total_count_non}")
    print("\t====================")





Processing for BARREL:

For data_A:


	For tt:
	Count of prompt: 19.33811983352914
	Count of nonprompt: 18.40767713672735
	Count of all: 37.745796970256485
	------
	For wz:
	Count of prompt: 13.494361915948277
	Count of nonprompt: 302.6347025863304
	Count of all: 316.12906450227877
	------
	For dy2:
	Count of prompt: 1834.2830092629508
	Count of nonprompt: 2035.863069135077
	Count of all: 3870.1460783980274
	------

For data_B:
	For tt:
	Count of prompt: 6.663981477093918
	Count of nonprompt: 43.02668795210451
	Count of all: 49.69066942919842
	------
	For wz:
	Count of prompt: 4.211616139372391
	Count of nonprompt: 81.00914717058122
	Count of all: 85.2207633099536
	------
	For dy2:
	Count of prompt: 600.9187566804145
	Count of nonprompt: 10584.38617688762
	Count of all: 11185.304933568035
	------

For data_C:
	For tt:
	Count of prompt: 0.12573549956780974
	Count of nonprompt: 0.10058839965424782
	Count of all: 0.22632389922205756
	------
	For wz:
	Count of prompt: 0.08595134978311003
	Count of nonprompt: 1.3322459216382054

In [7]:
import awkward as ak
import numpy as np

LUMI = {
    "2016pre": 19.52, "2016post": 16.81, "2017": 41.48, "2018": 59.83, "2022CD": 6.36,
    "2022FG": 21.6,"2022EFG": 39.22
}
XSEC = {
    'zg0': 124.6,
    'zg1': 1.74,
    'zg2': 0.30,
    'zg3': 0.042,
    'tt': 96.9,
    'ww': 116.8,
    'wz': 54.3,
    'zz': 16.7,
    'dy': 5558.0,
    'dy1':19317.5,
    'dy2':6346.0,
}
# #all
# A_prompt = 6481.6380519282875
# B_prompt = 1664.4474675365414
# C_prompt = 75.9491372036926
# D_prompt = 26.332492584624983
#end
# A_prompt = 1735.307602942849
# B_prompt = 463.3046431337174
# C_prompt = 55.456196229913594
# D_prompt = 21.18293413012198
#barrel
A_prompt = 4724.191909791624
B_prompt = 1249.6579314426272
C_prompt = 20.423137090023083
D_prompt = 5.054112003575507

keys = ['zg0', 'zg1', 'zg2','zg3','tt','zz','wz','ww','dy2']  #
# keys = ['tt','zz','wz','ww','dy']  #
# keys = ['zg1']  #
datasets = {'data_A': data_A, 'data_B': data_B, 'data_C': data_C, 'data_D': data_D}

for dataset_name, dataset in datasets.items():
    total_count_all = 0  # 初始化总计数器
    total_count_prompt = 0  # 初始化总计数器
    total_count_non = 0  # 初始化总计数器
    print(f"\nFor {dataset_name}:")

    # 根据数据集名称选择对应的prompt值
    if dataset_name == 'data_A':
        dataset_prompt = A_prompt
    elif dataset_name == 'data_B':
        dataset_prompt = B_prompt
    elif dataset_name == 'data_C':
        dataset_prompt = C_prompt
    elif dataset_name == 'data_D':
        dataset_prompt = D_prompt

    for key in keys:
        if len(dataset[key]['Photon_genPartFlav']) != 0:
            # masked_end_pt = data_A[key]['Photon_genPartFlav'][data_A[key]['photon_endcap']]
            # masked_end_pt = data_A[key]['Photon_genPartFlav'][data_A[key]['photon_barrel']]
            # masked_end_pt = dataset[key]['Photon_genPartFlav'][dataset[key]['photon_barrel']]
            masked_end_pt = dataset[key]['Photon_genPartFlav']
            filtered_end = ak.flatten(masked_end_pt)
            # ak.to_numpy(filtered_flav_pt)
            count_1 = np.sum(filtered_end == 1)
            count_not_1 = np.sum(filtered_end != 1)
            count_all = np.sum(filtered_end != -1)

            # numpy_arr = ak.to_numpy(dataset[key]['Photon_genPartFlav'])

            # count_1 = np.sum(numpy_arr == 1)
            # count_not_1 = np.sum(numpy_arr != 1)
            # count_all = np.sum(numpy_arr != -1)

            true_prompt = count_1 * XSEC[key] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(dataset[key]['event_weight']))
            true_non = count_not_1 * XSEC[key] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(dataset[key]['event_weight']))
            true_all = count_all * XSEC[key] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(dataset[key]['event_weight']))

            if 'dy' in key:  # 对于含 'dy' 的样本
                total_count_all += true_non
                total_count_non += true_non
            elif 'zg' in key:
                total_count_all += true_prompt
                total_count_prompt += true_prompt
            else:  # 对于其他样本
                total_count_all += true_all
                total_count_prompt += true_prompt
                total_count_non += true_non
            # total_count_non += true_non

            print(f"\tFor {key}:")
            print(f"\tCount of prompt: {true_prompt}")
            print(f"\tCount of nonprompt: {true_non}")
            print(f"\tCount of all: {true_all}")
            print("\t------")

    # 加入对应的A/B/C/D_prompt值
    # total_count_all += dataset_prompt
    # total_count_prompt += dataset_prompt

    # 打印该数据集的所有样本的 count_all 和 count_prompt 总和
    print(f"\tTotal count_all for {dataset_name}: {total_count_all}")
    print(f"\tTotal count_prompt for {dataset_name}: {total_count_prompt}")
    print(f"\tTotal count_non for {dataset_name}: {total_count_non}")
    print("\t====================")




For data_A:


	For zg0:
	Count of prompt: 3941.6909794272815
	Count of nonprompt: 35.578059545456185
	Count of all: 3977.269038972738
	------
	For zg1:
	Count of prompt: 1704.8797326060455
	Count of nonprompt: 15.268775381567234
	Count of all: 1720.1485079876131
	------
	For zg2:
	Count of prompt: 440.5365674013611
	Count of nonprompt: 4.7074254753484714
	Count of all: 445.24399287670957
	------
	For zg3:
	Count of prompt: 86.06551494515712
	Count of nonprompt: 0.9916498096518451
	Count of all: 87.05716475480897
	------
	For tt:
	Count of prompt: 23.96518621762454
	Count of nonprompt: 25.423718012611136
	Count of all: 49.388904230235674
	------
	For zz:
	Count of prompt: 8.012521696007935
	Count of nonprompt: 33.168113067195634
	Count of all: 41.18063476320357
	------
	For wz:
	Count of prompt: 18.78036992760954
	Count of nonprompt: 439.8560325150656
	Count of all: 458.6364024426751
	------
	For ww:
	Count of prompt: 8.44818156799138
	Count of nonprompt: 0.461649266010458
	Count of all: 8.9098308340

In [10]:
def load_event_dict(file_prefix, keys):
    data_subkeys = ['photon_pt', 'photon_eta', 'muon1_pt', 'muon2_pt', 'muon1_eta', 'muon2_eta', 'muon_mass','gmumu_mass','photon_endcap','photon_barrel']#'dr_lg',
    other_subkeys = data_subkeys + ['generator_weight', 'event_weight']
    
    event_dict = {}
    for key in keys:
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        event_dict[key] = {}
        for subkey in subkeys:
            event_dict[key][subkey] = ak.from_parquet(f'{file_prefix}_{key}_{subkey}.parquet')
    return event_dict

In [11]:
keys = ['zg0', 'zg1', 'zg2','zg3','dy','tt','zz','wz','ww']  
# 读取数据
event_A = load_event_dict('selection_file/event_final', keys)
event_B = load_event_dict('ABCD_file/B/event_final', keys)
event_C = load_event_dict('ABCD_file/C/event_final', keys)
event_D = load_event_dict('ABCD_file/D/event_final', keys)
# event_A = load_event_dict('test_file/A/event_final', keys)


### A

In [29]:
# 合并key
data_keys = ["dataF", "dataG"]
event_A["data"] = {}

info_keys = list(event_A["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_A[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_A["data"][info_key] = ak.concatenate(data_to_concatenate)

del event_A["dataF"]
del event_A["dataG"]

zg_keys = ["zg0", "zg1","zg2","zg3"]
event_A["zg"] = {}

info_keys = list(event_A["zg0"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "zg1", "zg2", "zg3", "zg4" 这四个 key 的数据
    zg_to_concatenate = [event_A[zg_key][info_key] for zg_key in zg_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_A["zg"][info_key] = ak.concatenate(zg_to_concatenate)


In [27]:

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    return ak.sum(ak.flatten(event_A[key][condition_key]))

# keys = ["data", "zg0", "zg1", "zg2", "zg3", "tt", "zz", "wz", "ww","dy","zg"]
keys = ["zg0", "zg1", "zg2", "zg3", "tt", "zz", "wz", "ww","dy"]
# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_A[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['A_' + key + '_endcap'] = endcap_counts[key]
    vars()['A_' + key + '_barrel'] = barrel_counts[key]


Total events:
zg0: 88701
zg1: 1082106
zg2: 27236
zg3: 30551
tt: 1430
zz: 197
wz: 4660
ww: 184
dy: 2520

For endcap:
zg0: 23669
zg1: 286991
zg2: 7330
zg3: 7484
tt: 338
zz: 77
wz: 1412
ww: 52
dy: 749

For barrel:
zg0: 64681
zg1: 790935
zg2: 19827
zg3: 22955
tt: 1085
zz: 119
wz: 3222
ww: 131
dy: 1762


### B

In [31]:
# 合并key
data_keys = ["dataF", "dataG"]
event_B["data"] = {}

info_keys = list(event_B["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_B[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_B["data"][info_key] = ak.concatenate(data_to_concatenate)

del event_B["dataF"]
del event_B["dataG"]

zg_keys = ["zg0", "zg1","zg2","zg3"]
event_B["zg"] = {}

info_keys = list(event_B["zg0"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "zg1", "zg2", "zg3", "zg4" 这四个 key 的数据
    zg_to_concatenate = [event_B[zg_key][info_key] for zg_key in zg_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_B["zg"][info_key] = ak.concatenate(zg_to_concatenate)

In [32]:

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    return ak.sum(ak.flatten(event_B[key][condition_key]))

# keys = ["data", "zg0", "zg1", "zg2", "zg3", "tt", "zz", "wz", "ww","dy"]
# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_B[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['B_' + key + '_endcap'] = endcap_counts[key]
    vars()['B_' + key + '_barrel'] = barrel_counts[key]


Total events:
data: 4007
zg0: 23006
zg1: 260764
zg2: 7485
zg3: 8461
tt: 492
zz: 59
wz: 1038
ww: 42
dy: 673
zg: 299716

For endcap:
data: 1574
zg0: 6196
zg1: 78166
zg2: 2015
zg3: 2097
tt: 104
zz: 16
wz: 329
ww: 13
dy: 188
zg: 88474

For barrel:
data: 2415
zg0: 16760
zg1: 211578
zg2: 5441
zg3: 6340
tt: 385
zz: 43
wz: 707
ww: 29
dy: 485
zg: 240119


### C

In [33]:
keys = ['dataF', 'dataG', 'zg0', 'zg1', 'zg2','zg3','dy','tt','zz','wz','ww'] 
# 读取数据
event_C = load_event_dict('ABCD_file/C/event_final', keys)
# event_C = load_event_dict('test_file/C/event_final', keys)

In [34]:
# 合并key
data_keys = ["dataF", "dataG"]
event_C["data"] = {}

info_keys = list(event_C["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_C[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_C["data"][info_key] = ak.concatenate(data_to_concatenate)

del event_C["dataF"]
del event_C["dataG"]

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    return ak.sum(ak.flatten(event_C[key][condition_key]))

keys = ["data", "zg0", "zg1", "zg2", "zg3", "tt", "zz", "wz", "ww","dy"]
# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_C[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['C_' + key + '_endcap'] = endcap_counts[key]
    vars()['C_' + key + '_barrel'] = barrel_counts[key]


Total events:
data: 107
zg0: 1586
zg1: 2273
zg2: 1
zg3: 6
tt: 20
zz: 4
wz: 60
ww: 2
dy: 55

For endcap:
data: 87
zg0: 1135
zg1: 2109
zg2: 0
zg3: 6
tt: 14
zz: 4
wz: 46
ww: 2
dy: 44

For barrel:
data: 20
zg0: 440
zg1: 392
zg2: 1
zg3: 0
tt: 6
zz: 0
wz: 14
ww: 0
dy: 11


### D

In [35]:
keys = ['dataF', 'dataG', 'zg0', 'zg1', 'zg2','zg3','dy','tt','zz','wz','ww'] 
# 读取数据
event_D = load_event_dict('ABCD_file/D/event_final', keys)

In [36]:
# 合并key
data_keys = ["dataF", "dataG"]
event_D["data"] = {}

info_keys = list(event_D["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_D[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_D["data"][info_key] = ak.concatenate(data_to_concatenate)

del event_D["dataF"]
del event_D["dataG"]

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    if key not in event_D:
        return 0
    if condition_key not in event_D[key]:
        return 0
    if len(event_D[key][condition_key]) == 0:
        return 0
    
    return ak.sum(ak.flatten(event_D[key][condition_key]))

keys = ["data", "zg0", "zg1", "zg2", "zg3", "tt", "zz", "wz", "ww","dy"]
# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_D[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['D_' + key + '_endcap'] = endcap_counts[key]
    vars()['D_' + key + '_barrel'] = barrel_counts[key]


Total events:
data: 279
zg0: 548
zg1: 863
zg2: 2
zg3: 0
tt: 11
zz: 2
wz: 14
ww: 2
dy: 17

For endcap:
data: 245
zg0: 434
zg1: 832
zg2: 2
zg3: 0
tt: 9
zz: 2
wz: 12
ww: 1
dy: 13

For barrel:
data: 34
zg0: 108
zg1: 123
zg2: 0
zg3: 0
tt: 2
zz: 0
wz: 2
ww: 1
dy: 4


# Method using NA_dataset

### DY & R

In [37]:
R_endcap = A_dy_endcap * D_dy_endcap / B_dy_endcap / C_dy_endcap
R_barrel = A_dy_barrel * D_dy_barrel / B_dy_barrel / C_dy_barrel

print('R_endcap',R_endcap)
print('R_barrel',R_barrel)

R_endcap 1.1771034816247583
R_barrel 1.321087160262418


### cc,cb,cd

In [39]:
LUMI = {"2016pre": 19.52, "2016post": 16.81, "2017": 41.48, "2018": 59.83, "2022CD": 6.36,"2022FG": 21.6,"2022EFG": 39.22}
XSEC = {
    # unit is pb^-1
    'zg0': 124.6,
    'zg1': 1.74,#27.11
    'zg2': 0.30,#0.88,
    'zg3': 0.042,#0.073,
    'tt': 96.9,#101.78,
    'ww': 116.8,#80.27,
    'wz': 54.3,#29.14,
    'zz': 16.7,#12.83,
}


def compute_truenum(event, region, suffix, LUMI, XSEC, considered_keys):
    truenum_dict = {}
    for key in considered_keys:
        variable_name = f"{region}_{key}_{suffix}"  # 如 A_zg0_barrel
        event_count = globals()[variable_name]
        truenum_dict[key] = event_count * XSEC[key] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(event[key]['event_weight']))
    return truenum_dict

considered_keys = [i for i in XSEC if (i != 'data') & (i != 'dy') & (i != 'zg')]
# For A:
A_truenum_barrel = compute_truenum(event_A, "A", "barrel", LUMI, XSEC, considered_keys)
A_truenum_endcap = compute_truenum(event_A, "A", "endcap", LUMI, XSEC, considered_keys)

A_prompt_barrel = sum(abs(A_truenum_barrel[key]) for key in considered_keys)
A_bkg_prompt_barrel = sum(abs(A_truenum_barrel[key]) for key in ["tt", "ww", "wz", "zz"])

A_prompt_endcap = sum(abs(A_truenum_endcap[key]) for key in considered_keys)
A_bkg_prompt_endcap = sum(abs(A_truenum_endcap[key]) for key in ["tt", "ww", "wz", "zz"])

print('A_prompt_barrel', A_prompt_barrel)
print('A_bkg_prompt_barrel', A_bkg_prompt_barrel)
print('A_prompt_endcap', A_prompt_endcap)
print('A_bkg_prompt_endcap', A_bkg_prompt_endcap)

# For A:
B_truenum_barrel = compute_truenum(event_B, "B", "barrel", LUMI, XSEC, considered_keys)
B_truenum_endcap = compute_truenum(event_B, "B", "endcap", LUMI, XSEC, considered_keys)

B_prompt_barrel = sum(abs(B_truenum_barrel[key]) for key in considered_keys)
B_bkg_prompt_barrel = sum(abs(B_truenum_barrel[key]) for key in ["tt", "ww", "wz", "zz"])

B_prompt_endcap = sum(abs(B_truenum_endcap[key]) for key in considered_keys)
B_bkg_prompt_endcap = sum(abs(B_truenum_endcap[key]) for key in ["tt", "ww", "wz", "zz"])

print('B_prompt_barrel', B_prompt_barrel)
print('B_bkg_prompt_barrel', B_bkg_prompt_barrel)
print('B_prompt_endcap', B_prompt_endcap)
print('B_bkg_prompt_endcap', B_bkg_prompt_endcap)


# For A:
C_truenum_barrel = compute_truenum(event_C, "C", "barrel", LUMI, XSEC, considered_keys)
C_truenum_endcap = compute_truenum(event_C, "C", "endcap", LUMI, XSEC, considered_keys)

C_prompt_barrel = sum(abs(C_truenum_barrel[key]) for key in considered_keys)
C_bkg_prompt_barrel = sum(abs(C_truenum_barrel[key]) for key in ["tt", "ww", "wz", "zz"])

C_prompt_endcap = sum(abs(C_truenum_endcap[key]) for key in considered_keys)
C_bkg_prompt_endcap = sum(abs(C_truenum_endcap[key]) for key in ["tt", "ww", "wz", "zz"])

print('C_prompt_barrel', C_prompt_barrel)
print('C_bkg_prompt_barrel', C_bkg_prompt_barrel)
print('C_prompt_endcap', C_prompt_endcap)
print('C_bkg_prompt_endcap', C_bkg_prompt_endcap)

# For A:
D_truenum_barrel = compute_truenum(event_D, "D", "barrel", LUMI, XSEC, considered_keys)
D_truenum_endcap = compute_truenum(event_D, "D", "endcap", LUMI, XSEC, considered_keys)

D_prompt_barrel = sum(abs(D_truenum_barrel[key]) for key in considered_keys)
D_bkg_prompt_barrel = sum(abs(D_truenum_barrel[key]) for key in ["tt", "ww", "wz", "zz"])

D_prompt_endcap = sum(abs(D_truenum_endcap[key]) for key in considered_keys)
D_bkg_prompt_endcap = sum(abs(D_truenum_endcap[key]) for key in ["tt", "ww", "wz", "zz"])

print('D_prompt_barrel', D_prompt_barrel)
print('D_bkg_prompt_barrel', D_bkg_prompt_barrel)
print('D_prompt_endcap', D_prompt_endcap)
print('D_bkg_prompt_endcap', D_bkg_prompt_endcap)

A_prompt_barrel 4721.948750231798
A_bkg_prompt_barrel 182.88692726625064
A_prompt_endcap 1735.935279248449
A_bkg_prompt_endcap 78.75595088454901
B_prompt_barrel 1249.6579314426272
B_bkg_prompt_barrel 45.410448625775594
B_prompt_endcap 463.3046431337174
B_bkg_prompt_endcap 18.845132891691144
C_prompt_barrel 20.423137090023083
C_bkg_prompt_barrel 0.7525415693858519
C_prompt_endcap 55.456196229913594
C_bkg_prompt_endcap 2.793944608044445
D_prompt_barrel 5.054112003575507
D_bkg_prompt_barrel 0.18241031668551638
D_prompt_endcap 21.18293413012198
D_bkg_prompt_endcap 0.9745339205164781


In [42]:
c_B_barrel = B_prompt_barrel/A_prompt_barrel
c_C_barrel = C_prompt_barrel/A_prompt_barrel
c_D_barrel = D_prompt_barrel/A_prompt_barrel

print('c_B_barrel', c_B_barrel)
print('c_C_barrel', c_C_barrel)
print('c_D_barrel', c_D_barrel)

c_B_endcap = B_prompt_endcap/A_prompt_endcap
c_C_endcap = C_prompt_endcap/A_prompt_endcap
c_D_endcap = D_prompt_endcap/A_prompt_endcap

print('c_B_endcap', c_B_endcap)
print('c_C_endcap', c_C_endcap)
print('c_D_endcap', c_D_endcap)

c_B_barrel 0.26464877056983777
c_C_barrel 0.004325150095926078
c_D_barrel 0.0010703445274215235
c_B_endcap 0.2668905048892717
c_C_endcap 0.03194600449270358
c_D_endcap 0.012202605928541794


## data

In [43]:
print('A_data_barrel',A_data_barrel)
print('B_data_barrel',B_data_barrel)
print('C_data_barrel',C_data_barrel)
print('D_data_barrel',D_data_barrel)
print('A_data_endcap',A_data_endcap)
print('B_data_endcap',B_data_endcap)
print('C_data_endcap',C_data_endcap)
print('D_data_endcap',D_data_endcap)

A_data_barrel 2621
B_data_barrel 2415
C_data_barrel 20
D_data_barrel 34
A_data_endcap 1471
B_data_endcap 1574
C_data_endcap 87
D_data_endcap 245


### calculate NA(nonprompt)

In [10]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NA_prompt, NB_bkg, NC, cC, ND, cD, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NA_prompt NB_bkg NC cC ND cD NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))

# 已知的值
values_barrel = {
    NA:7113.929840399236,  # A_data allv2 7145.689 纯dy 3870.1460 
    R: 1.3210,  # 
    NB: 11968.9555410694,  # B_data allv2 7145.689 纯dy 3870.1460 
    cB: 0.264648770,  # 
    NC: 61.237243354302386,  # C_data
    cC: 0.0043251,  # 
    ND: 211.85464293244655,  # D_data
    cD: 0.0010703,  # 
    NA_bkg: 182.88692,
    NB_bkg: 45.410448625775594,  
    NC_bkg: 0.7525415693858519,  
    ND_bkg: 0.18241031668551638,  
}


# 解方程
solution = sp.solve(equation.subs(values_barrel), NA_prompt)

# 输出解
print(f"NA_prompt_barrel = {solution[0]}")
print(f"NA_prompt_barrel = {solution[1]}")
NA_prompt_barrel = solution[1]


# #endcap
# values_endcap = {
#     NA: 10054.015809471077,  # A_data 4924.0358 11405.63 Allv2 10362.481
#     R: 1.177103,  
#     NB: 16499.700281056892,  # B_data 1574 6751.76 Allv2 16594.442 
#     cB: 0.2668905,  # 
#     NC:  326.5116886905495,  # C_data 87 233.9 Allv2 330.4875
#     cC: 0.0319460,  # 
#     ND: 1157.6410789087572,  # D_data 245 382.22 Allv2 1159.138
#     cD: 0.012202605,  # 
#     NA_bkg: 78.75595088454901,
#     NB_bkg: 18.845132891691144,  # B_dy
#     NC_bkg: 2.793944608044445,  # C_dy
#     ND_bkg: 0.9745339205164781,
# }

#all
values_endcap = {
    NA: 10054.015809471077,  # A_data 4924.0358 11405.63 Allv2 10362.481
    R: 1.157,  
    NB: 16499.700281056892,  # B_data 1574 6751.76 Allv2 16594.442 
    cB: 0.268,  # 
    NC:  326.5116886905495,  # C_data 87 233.9 Allv2 330.4875
    cC: 0.0035,  # 
    ND: 1157.6410789087572,  # D_data 245 382.22 Allv2 1159.138
    cD: 0.0013,  # 
    NA_bkg: 263.0754951199856,
    NB_bkg: 64.41697392770189,  # B_dy
    NC_bkg: 3.5464861774302974,  # C_dy
    ND_bkg: 1.1569442372019945,
}

# 解方程
solution = sp.solve(equation.subs(values_endcap), NA_prompt)

# 输出解
print(f"NA_prompt_endcap = {solution[0]}")
print(f"NA_prompt_endcap = {solution[1]}")
NA_prompt_endcap = solution[0]



NA_prompt_barrel = -297781.647256804
NA_prompt_barrel = 3910.47550373543
NA_prompt_endcap = 5174.42401978399
NA_prompt_endcap = 4663457.08842603


In [12]:
5174.424+263.0754

5437.4994

In [11]:
# N_nonprompt_barrel = A_data_barrel - A_bkg_prompt_barrel - NA_prompt_barrel
#allv3
N_nonprompt_endcap = 10054.015809471077 - 263.0754 - 5174.42401978399
#allv2
# N_nonprompt_endcap = 10362.481 - 263.0754 - 5425.5473
#barrel
# N_nonprompt_barrel = 7113.929840399236 - 182.88692 - 3910.47550373543
# #endcap
# N_nonprompt_endcap = 3165.563790954001 - 78.75595088454901 - 2063.82177778520
# print('N_nonprompt_barrel = ',N_nonprompt_barrel)
print('N_nonprompt_endcap = ',N_nonprompt_endcap)

N_nonprompt_endcap =  4616.516389687087


In [19]:
3020.5674166638064+182.88692

3203.4543366638063

In [ ]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NC, cC, ND, cD, NB_bkg, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NC cC ND cD NB_bkg NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))

# 已知的值
values = {
    NA: 3977.269038972738,  # A_data
    R: 1.157,  # 
    NB: 4007,  # B_data
    cB: 0.268,  # 
    NC: 107,  # C_data
    cC: 0.0035,  # 
    ND: 279,  # D_data
    cD: 0.0013,  # 
    NA_bkg: 263.0754951199856,
    NB_bkg: 64.41697392770189,  # B_dy
    NC_bkg: 3.5464861774302974,  # C_dy
    ND_bkg: 1.1569442372019945,  # D_dy
}

# 解方程得到NA_prompt的值
NA_prompt_value = sp.solve(equation.subs(values), NA_prompt)[0]

# 计算公式【1】中每个变量的偏导数
partial_derivatives = {
    var: equation.lhs.diff(var) for var in [R, NB, cB, NC, cC, ND, cD]
}

# 计算每个变量的不确定度
uncertainties = {
    R: 0.2845 * values[R],
    NB: sp.sqrt(values[NB]),
    cB: sp.sqrt((sp.sqrt(NB)/NB)**2 + (sp.sqrt(NA)/NA)**2).subs(values),
    NC: sp.sqrt(values[NC]),
    cC: sp.sqrt((sp.sqrt(NC)/NC)**2 + (sp.sqrt(NC)/NC)**2).subs(values),
    ND: sp.sqrt(values[ND]),
    cD: sp.sqrt((sp.sqrt(ND)/ND)**2 + (sp.sqrt(ND)/ND)**2).subs(values),
}

# 使用误差传递公式计算NA_prompt的不确定度
sigma_NA_prompt_squared = sum([
    (partial_derivatives[var].subs(NA_prompt, NA_prompt_value) * uncertainties[var])**2 for var in partial_derivatives
])

sigma_NA_prompt = sp.sqrt(sigma_NA_prompt_squared)

# 使用公式【2】计算N_nonprompt的不确定度
sigma_N_nonprompt = sp.sqrt(values[NA] + values[NA_bkg] + sigma_NA_prompt**2)

print(f"NA_prompt uncertainty = {sigma_NA_prompt}")
print(f"N_nonprompt uncertainty = {sigma_N_nonprompt}")

# 解方程
solution = sp.solve(equation.subs(values), NA_prompt)

# 输出解
# print(f"NA_prompt = {solution[0]}")
# print(f"NA_prompt = {solution[1]}")
NA_prompt = solution[0]
N_nonprompt = A_data - A_bkg_prompt - NA_prompt
print("NA_nonprompt =",N_nonprompt)


NA_prompt uncertainty = 0
N_nonprompt uncertainty = 66.0762854216245
NA_nonprompt = 1299.91483599553
